In [9]:
using QuantumCumulants
using IterTools
using Symbolics
using SymbolicUtils
using SymbolicUtils.Rewriters
using OrdinaryDiffEq, ModelingToolkit
using Plots

include("../src/diagrams.jl")
include("../src/contractions.jl")
include("../src/lindblad.jl")
include("../src/printing.jl")
include("../src/poles.jl")

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **

ERROR: LoadError: UndefVarError: _unwrap_val not defined
Stacktrace:
 [1] include
   @ ./Base.jl:418 [inlined]
 [2] include_package_for_output(pkg::Base.PkgId, input::String, depot_path::Vector{String}, dl_load_path::Vector{String}, load_path::Vector{String}, concrete_deps::Vector{Pair{Base.PkgId, UInt64}}, source::String)
   @ Base ./loading.jl:1318
 [3] top-level scope
   @ none:1
 [4] eval
   @ ./boot.jl:373 [inlined]
 [5] eval(x::Expr)
   @ Base.MainInclude ./client.jl:453
 [6] top-level scope
   @ none:1
in expression starting at /Users/leonbello/.julia/packages/

ErrorException: Failed to precompile OrdinaryDiffEq [1dea7af3-3e70-54e6-95c3-0bf5283fa5ed] to /Users/leonbello/.julia/compiled/v1.7/OrdinaryDiffEq/jl_pTLf0t.

In [288]:
node = DiagramNode((1,2))
ds = get_diagrams(node)

4-element Vector{Any}:
 [(1, 2)]
 [(0, 1), (1, 1)]
 [(0, 2), (1, 0)]
 [(0, 1), (0, 1), (1, 0)]

In [239]:
[(2, 0), (1, 0)] == [(1, 0), (2, 0)]

false

In [158]:
for d in ds 
    print(getindex(d))
end

BoundsError: BoundsError: attempt to access 2-element Vector{Tuple{Int64, Int64}} at index []

In [180]:
pushfirst!(filter!(x-> ((0,0) ∉ x), ds[1:length(ds)]),ds[1])

4-element Vector{Any}:
 [(0, 0), (3, 0)]
 [(1, 0), (2, 0)]
 [(2, 0), (1, 0)]
 [(1, 0), (1, 0), (1, 0)]

In [217]:
h_cav = FockSpace(:cavity)
h_atom = NLevelSpace(:atom, (:g,:e))
h = tensor(h_cav, h_atom)

@qnumbers a::Destroy(h) σ::Transition(h)
@cnumbers ω_0 ω_d g κ γ ϵ ω_3 ω_1

σz = 2* σ(:e, :e)  - 1
σx = σ(:g, :e) + σ(:e, :g)
σy = 1im* (σ(:e, :g) - σ(:g, :e))
@syms t::Real
H_rab = ω_0 * a'*a + ω_d/2 * σz + g*(a' + a)*(σx) + ϵ*(a*exp(1im*ω_3*t) + a'*exp(-1im*ω_3*t))
#@register f(t)
ωs = [0,0,0,ω_3, -ω_3]
hs = [ω_0 * a'*a,ω_d/2 * σz,g*(a' + a)*(σx),ϵ*a, ϵ*a']

ωs2 = [ω_1,-ω_1,ω_3, -ω_3]
hs2 = [0.5*ω_0 * a'*a,0.5*ω_0 * a'*a,ϵ*a, ϵ*a']

4-element Vector{QuantumCumulants.QMul{Nothing}}:
 0.5ω_0*(a′*a)
 0.5ω_0*(a′*a)
 ϵ*(a)
 ϵ*(a′)

In [203]:
k=1
ω_list = repeated_combinations(ωs2, k)
h_list = repeated_combinations(hs2, k)
print(h_list)

Vector{QuantumCumulants.QMul{Nothing}}[[0.5ω_0*(a′*a)], [0.5ω_0*(a′*a)], [ϵ*(a)], [ϵ*(a′)]]

In [4]:
h_eff = 0
g = 0
V = 0
g_list, V_list, freq_list = [],[],[]               
for (ω, h) in zip(freq_list, V_list)
    #print(h)
    #println(h)
    g = (contraction_coeff((k,0),ω)) #+ contraction_coeff((k,0),-reverse(ω)))#*exp(1im*sum(ω)*t)
    #print(g)
    V = h[1] #? 
    push!(g_list, g)
    push!(V_list, V)
    #println(g)
    #print(h)
    print(g)
    push!(freq_list, sum(ω))
    h_eff += g*V
end

In [8]:
@cnumbers ω_1 ω_2 ω_3
#simplify(contraction_coeff((3,0),[ω_1,-ω_1,ω_2]))
c, cs = contraction_coeff((1,2),[ω_1,ω_2,ω_3])
print(5)

UndefVarError: UndefVarError: contraction_coeff not defined

In [287]:
cs[4]

((exp(-0.5(τ^2)*(ω_1^2))*exp(-0.5(τ^2)*((ω_2 + ω_3)^2))) / (ω_1*ω_3*(ω_2 + ω_3)), [(0, 1), (2, 0)])

In [269]:
sum(cs) - c

0

In [290]:
c

(-exp(-0.5(τ^2)*(ω_3^2))*exp(-0.5(τ^2)*((ω_1 + ω_2)^2))) / (ω_1*ω_3*(ω_1 + ω_2)) + (-exp(-0.5(τ^2)*(ω_1^2))*exp(-0.5(τ^2)*((ω_2 + ω_3)^2))) / (ω_1*ω_2*ω_3) + (exp(-0.5(τ^2)*(ω_1^2))*exp(-0.5(τ^2)*(ω_2^2))*exp(-0.5(τ^2)*(ω_3^2))) / (ω_1*ω_2*ω_3) + exp(-0.5(τ^2)*((ω_1 + ω_2 + ω_3)^2)) / (ω_2*(ω_2 + ω_3))

In [301]:
diagram = [(0,1),(2,0)]
freqs = [ω_1,ω_2,ω_3]
μ, ν = [],[]
ind_μ = 0
ind_ν = 0
ububs, dbubs = get_max_modes(diagram)
freqs_up = reverse(freqs[1:ububs])
freqs_dn = freqs[ububs+1:ububs+dbubs]
for (i, bubble) in enumerate(diagram)
    (μ_len, ν_len) = bubble
    push!(μ, freqs_up[ind_μ+1:ind_μ+μ_len])
    ind_μ += μ_len
    push!(ν,freqs_dn[ind_ν+1:ind_ν+ν_len]) 
    ind_ν += ν_len
end
ω = tuple.(μ, ν)

2-element Vector{Tuple{Vector{Sym{Parameter, Base.ImmutableDict{DataType, Any}}}, Vector{Sym{Parameter, Base.ImmutableDict{DataType, Any}}}}}:
 ([], [ω_1])
 ([ω_2, ω_3], [])

2-element Vector{Sym{Parameter, Base.ImmutableDict{DataType, Any}}}:
 ω_2
 ω_1

In [298]:
freqs[1:1]

1-element Vector{Sym{Parameter, Base.ImmutableDict{DataType, Any}}}:
 ω_1

In [250]:
k=2
h_eff = effective_hamiltonian(k, ωs2, hs2)
H_eff = simplify(h_eff)
secondOrderRule = @acrule((~a)*(~y)*(~z) + (~b)*(~y)*(~z) => ((~a)+(~b))*(~y)*(~z))
firstOrderRule = @acrule((~a)*(~y) + (~b)*(~y) => ((~a)+(~b))*(~y))
rules = Chain([firstOrderRule,secondOrderRule])#, addRule])
H1 = Fixpoint(rules)(H_eff)

MethodError: MethodError: no method matching +(::Tuple{Term{CNumber, Nothing}, Vector{Any}}, ::Tuple{Term{CNumber, Nothing}, Vector{Any}})
Closest candidates are:
  +(::Any, ::Any, !Matched::Any, !Matched::Any...) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/operators.jl:655
  +(!Matched::Union{InitialValues.NonspecificInitialValue, InitialValues.SpecificInitialValue{typeof(+)}}, ::Any) at ~/.julia/packages/InitialValues/OWP8V/src/InitialValues.jl:154
  +(!Matched::MultivariatePolynomials.RationalPoly, ::Any) at ~/.julia/packages/MultivariatePolynomials/1bIGc/src/rational.jl:50
  ...

In [ ]:
simplify(substitute())

In [6]:
#substitute(H1, Dict([f(j,t)=> exp(1im*freqs[j]*t)]))

UndefVarError: UndefVarError: f not defined

In [36]:
k=2
@syms t::Real
h_eff, freqs = effective_hamiltonian(k, ωs, hs)

((Inf*ω_0*f(1, t)*(a′*a)+0+Inf*ω_d*f(2, t)*(σee)+-Inf*ω_d*f(2, t)+Inf*g*f(3, t)*(a′*σge)+Inf*g*f(3, t)*(a′*σeg)+Inf*g*f(3, t)*(a*σge)+Inf*g*f(3, t)*(a*σeg)+ϵ*(((-1//2)*exp(-0.5(τ^2)*(ω_3^2))) / ω_3 + ((1//2)*exp(-0.5(τ^2)*(ω_3^2))) / ω_3)*f(4, t)*(a)+ϵ*(((-1//2)*exp(-0.5(τ^2)*(ω_3^2))) / ω_3 + ((1//2)*exp(-0.5(τ^2)*(ω_3^2))) / ω_3)*f(5, t)*(a′)+(ω_0^2)*(NaN - 3.0(τ^2))*f(6, t)*(a′*a′*a*a)+(ω_0^2)*(NaN - 3.0(τ^2))*f(6, t)*(a′*a)+ω_0*ω_d*(NaN - 3.0(τ^2))*f(7, t)*(a′*a*σee)+(-1//2)*ω_0*ω_d*(NaN - 3.0(τ^2))*f(7, t)*(a′*a)+g*ω_0*(NaN - 3.0(τ^2))*f(8, t)*(a′*a′*a*σge)+g*ω_0*(NaN - 3.0(τ^2))*f(8, t)*(a′*a′*a*σeg)+g*ω_0*(NaN - 3.0(τ^2))*f(8, t)*(a′*a*a*σge)+g*ω_0*(NaN - 3.0(τ^2))*f(8, t)*(a*σge)+g*ω_0*(NaN - 3.0(τ^2))*f(8, t)*(a′*a*a*σeg)+g*ω_0*(NaN - 3.0(τ^2))*f(8, t)*(a*σeg)+ω_0*ϵ*((-0.5exp(-0.5(τ^2)*(ω_3^2))) / ω_3 + ((1//2)*exp(-0.5(τ^2)*(ω_3^2))) / ω_3 + ((1//2)*exp(-0.5(τ^2)*(ω_3^2))) / (ω_3^2) + (1.5(3.0 + -2 / ω_3 - 2.0ω_3*(τ^2))*exp(-0.5(τ^2)*(ω_3^2))) / ω_3 - 103.5exp(-0.5(τ^2)*(ω_3^

In [25]:

H_eff = simplify(h_eff)
secondOrderRule = @acrule((~a)*(~y)*(~z) + (~b)*(~y)*(~z) => ((~a)+(~b))*(~y)*(~z))
firstOrderRule = @acrule((~a)*(~y) + (~b)*(~y) => ((~a)+(~b))*(~y))
rules = Chain([firstOrderRule,secondOrderRule])
H1 = Fixpoint(rules)(H_eff)

((-10.0ω_d*ϵ*exp(-0.5(τ^2)*(ω_3^2))*f(14, t)) / (4(ω_3^2)) + (-10.0ω_d*ϵ*exp(-0.5(τ^2)*(ω_3^2))*f(22, t)) / (4(ω_3^2)) + (18.0ω_d*ϵ*exp(-0.5(τ^2)*(ω_3^2))*f(14, t)) / (4ω_3) + (-414.0ω_d*ϵ*exp(-0.5(τ^2)*(ω_3^2))*f(14, t)) / 4 + (-18.0ω_d*ϵ*exp(-0.5(τ^2)*(ω_3^2))*f(22, t)) / (4ω_3) + (-414.0ω_d*ϵ*exp(-0.5(τ^2)*(ω_3^2))*f(22, t)) / 4 + (-12.0ω_d*ϵ*(τ^2)*exp(-0.5(τ^2)*(ω_3^2))*f(14, t)) / 4 + (-12.0ω_d*ϵ*(τ^2)*exp(-0.5(τ^2)*(ω_3^2))*f(22, t)) / 4*(a*σee)+(-10.0ω_d*ϵ*exp(-0.5(τ^2)*(ω_3^2))*f(15, t)) / (4(ω_3^2)) + (-10.0ω_d*ϵ*exp(-0.5(τ^2)*(ω_3^2))*f(27, t)) / (4(ω_3^2)) + (-18.0ω_d*ϵ*exp(-0.5(τ^2)*(ω_3^2))*f(15, t)) / (4ω_3) + (18.0ω_d*ϵ*exp(-0.5(τ^2)*(ω_3^2))*f(27, t)) / (4ω_3) + (-414.0ω_d*ϵ*exp(-0.5(τ^2)*(ω_3^2))*f(15, t)) / 4 + (-414.0ω_d*ϵ*exp(-0.5(τ^2)*(ω_3^2))*f(27, t)) / 4 + (-12.0ω_d*ϵ*(τ^2)*exp(-0.5(τ^2)*(ω_3^2))*f(15, t)) / 4 + (-12.0ω_d*ϵ*(τ^2)*exp(-0.5(τ^2)*(ω_3^2))*f(27, t)) / 4*(a′*σee)+(-36.0g*f(3, t)) / 4 + (378.0g*ω_d*f(13, t)) / 4 + (-378.0g*ω_d*f(17, t)) / 4 + (-756.0g

In [27]:
eqs = meanfield([a] , H1; order=1)
eqs = complete(eqs)

DivideError: DivideError: integer division error

In [28]:
#H_test = σ(1,2,2)⊗σ(2,1,1) #9*(ω_1*σ(2,2,1)+ω_2*σ(2,2,2) - g*σ(1,2,1)*σ(2,1,2))# -  g*σ(1,2,1)*σ(2,2,1) 
            #-  g*σ(1,1,2)*σ(2,1,2)- g*σ(1,1,2)*σ(2,2,1)) - 4.5*(ω_1 + ω_2)
            
#H_test2 = 9*ω_1*σ(2,2,1) - 4.5*ω_1 + ϵ_d*exp(1im*ω_d)/(2*ω_d^2)  +σ(1,2,1) + σ(2,1,1) # (ϵ_d/(2*ω_d^2))*σ(1,2,1) #+ (ϵ_d/(2*ω_d^2))*σ(2,1,1)
@register f(t)
#f(t) = exp(1im*ω_d*t)
H_rab = ω_0 * a'*a + ω_d/2 * σz + g*(a' + a)*(σx) + f(t)*(a + a')


(ω_d*(σee)+(-1//2)*ω_d+ω_0*(a′*a)+g*(a′*σge)+g*(a′*σeg)+g*(a*σge)+g*(a*σeg)+f(t)*(a)+f(t)*(a′))

In [33]:
eqs = meanfield([a] , H_rab; order=3, iv=t)
eqs = complete(eqs)

∂ₜ(⟨a⟩) = (0 - 1im)*f(t) + g*((0 - 1im)*⟨σeg⟩ + (0 - 1im)*⟨σge⟩) + (0 - 1im)*ω_0*⟨a⟩
∂ₜ(⟨σeg⟩) = g*((0 + 1im)*⟨a′⟩ + (0 + 1im)*⟨a⟩ + (0 - 2im)*⟨a′*σee⟩ + (0 - 2im)*⟨a*σee⟩) + (0 + 1im)*ω_d*⟨σeg⟩
∂ₜ(⟨a′*σee⟩) = g*((0 + 1im)*⟨σge⟩ + (0 - 1im)*⟨a′*a′*σeg⟩ + (0 + 1im)*⟨a′*a′*σge⟩ + (0 - 1im)*⟨a′*a*σeg⟩) + (0 + 1im)*g*⟨a′*a*σge⟩ + (0 + 1im)*ω_0*⟨a′*σee⟩ + (0 + 1im)*f(t)*⟨σee⟩
∂ₜ(⟨a′*a′*σeg⟩) = g*((0 + 1im)*⟨a′*a′*a′⟩ + (0 - 2im)*⟨a′*σee⟩ + (0 + 1im)*⟨a′*a′*a⟩ + (0 - 2im)*⟨σee⟩*⟨a′*a′*a⟩ + (0 + 4im)*(⟨a′⟩^2)*⟨a*σee⟩ + (0 - 4im)*⟨a′⟩*⟨a′*a*σee⟩ + (0 - 2im)*⟨a⟩*⟨a′*a′*σee⟩ + (0 - 2im)*⟨a′*a′⟩*⟨a*σee⟩ + (0 - 4im)*⟨a′*a⟩*⟨a′*σee⟩ + (0 + 8im)*⟨a′⟩*⟨a⟩*⟨a′*σee⟩ + (0 + 4im)*⟨a⟩*⟨σee⟩*⟨a′*a′⟩ + (0 + 8im)*⟨a′⟩*⟨σee⟩*⟨a′*a⟩ + (0 - 12im)*(⟨a′⟩^2)*⟨a⟩*⟨σee⟩) + (0 - 2im)*g*(⟨σee⟩*⟨a′*a′*a′⟩ + 3⟨a′*a′⟩*⟨a′*σee⟩ + 3⟨a′⟩*⟨a′*a′*σee⟩ + 6(⟨a′⟩^3)*⟨σee⟩ - 6(⟨a′⟩^2)*⟨a′*σee⟩ - 6⟨a′⟩*⟨σee⟩*⟨a′*a′⟩) + (0 + 2im)*g*⟨a′⟩ + (0 + 2im)*ω_0*⟨a′*a′*σeg⟩ + (0 + 2im)*f(t)*⟨a′*σeg⟩ + (0 + 1im)*ω_d*⟨a′*a′*σeg⟩
∂ₜ(⟨a′*a′*σge⟩

In [34]:
substitute(eqs, Dict([f(t) => exp(1im*ω_d*t)]))

∂ₜ(⟨a⟩) = (0 - 1im)*exp((0 + 1im)*t*ω_d) + g*((0 - 1im)*⟨σeg⟩ + (0 - 1im)*⟨σge⟩) + (0 - 1im)*ω_0*⟨a⟩
∂ₜ(⟨σeg⟩) = g*((0 + 1im)*⟨a′⟩ + (0 + 1im)*⟨a⟩ + (0 - 2im)*⟨a′*σee⟩ + (0 - 2im)*⟨a*σee⟩) + (0 + 1im)*ω_d*⟨σeg⟩
∂ₜ(⟨a′*σee⟩) = g*((0 + 1im)*⟨σge⟩ + (0 - 1im)*⟨a′*a′*σeg⟩ + (0 + 1im)*⟨a′*a′*σge⟩ + (0 - 1im)*⟨a′*a*σeg⟩) + (0 + 1im)*g*⟨a′*a*σge⟩ + (0 + 1im)*ω_0*⟨a′*σee⟩ + (0 + 1im)*⟨σee⟩*exp((0 + 1im)*t*ω_d)
∂ₜ(⟨a′*a′*σeg⟩) = g*((0 + 1im)*⟨a′*a′*a′⟩ + (0 - 2im)*⟨a′*σee⟩ + (0 + 1im)*⟨a′*a′*a⟩ + (0 - 2im)*⟨σee⟩*⟨a′*a′*a⟩ + (0 + 4im)*(⟨a′⟩^2)*⟨a*σee⟩ + (0 - 4im)*⟨a′⟩*⟨a′*a*σee⟩ + (0 - 2im)*⟨a⟩*⟨a′*a′*σee⟩ + (0 - 2im)*⟨a′*a′⟩*⟨a*σee⟩ + (0 - 4im)*⟨a′*a⟩*⟨a′*σee⟩ + (0 + 8im)*⟨a′⟩*⟨a⟩*⟨a′*σee⟩ + (0 + 4im)*⟨a⟩*⟨σee⟩*⟨a′*a′⟩ + (0 + 8im)*⟨a′⟩*⟨σee⟩*⟨a′*a⟩ + (0 - 12im)*(⟨a′⟩^2)*⟨a⟩*⟨σee⟩) + (0 - 2im)*g*(⟨σee⟩*⟨a′*a′*a′⟩ + 6(⟨a′⟩^3)*⟨σee⟩ + 3⟨a′*a′⟩*⟨a′*σee⟩ + 3⟨a′⟩*⟨a′*a′*σee⟩ - 6(⟨a′⟩^2)*⟨a′*σee⟩ - 6⟨a′⟩*⟨σee⟩*⟨a′*a′⟩) + (0 + 2im)*g*⟨a′⟩ + (0 + 2im)*ω_0*⟨a′*a′*σeg⟩ + (0 + 2im)*⟨a′*σeg⟩*exp((0 + 1im)*t

In [49]:
k=1
γ, J_ops, Jd_ops = effective_dissipator(k, ωs, hs,t)
#print(5)

(Any[], Any[], Any[])

In [26]:
f(t) = exp(1im*ω_d*t)
@named sys = ODESystem(eqs)
#ω_0,ω_d, g = 1,1,1
vals = [1.0;1.0;1.0]
u0 = zeros(ComplexF64, length(eqs))
prob = ODEProblem(sys,u0,(0.0, 1), [g; ω_0; ω_d].=>vals)
sol = solve(prob,Tsit5(),maxiters=1e7)

ArgumentError: ArgumentError: Sym{Parameter, Base.ImmutableDict{DataType, Any}}[g, ω_0, ω_d] are missing from the variable map.

In [24]:
length(Jd_ops)

275

In [39]:
#Get meanfield equations using QuantumCumulants
eqs = meanfield([σz1, σz2, σx1, σx2, σx1*σz2] , H1; order=2,iv=t)

∂ₜ(1 - 2⟨σ_{1}22⟩) = ((0.0 - 4608.0im)*g*(ω_d^8)*⟨σ_{1}21*σ_{2}12⟩ + (0.0 - 4608.0im)*g*(ω_d^8)*⟨σ_{1}21*σ_{2}21⟩ + (0.0 + 4608.0im)*g*(ω_d^8)*⟨σ_{1}12*σ_{2}12⟩ + (0.0 + 4608.0im)*g*(ω_d^8)*⟨σ_{1}12*σ_{2}21⟩ + (0.0 - 2560.0im)*g*ϵ_d*(ω_d^6)*⟨σ_{1}21⟩*exp((0 + 1im)*t*ω_d - 0.5(τ^2)*(ω_d^2)) + (0.0 - 2560.0im)*g*ϵ_d*(ω_d^6)*⟨σ_{1}21⟩*exp((0 - 1im)*t*ω_d - 0.5(τ^2)*(ω_d^2)) + (0.0 + 2560.0im)*g*ϵ_d*(ω_d^6)*⟨σ_{1}12⟩*exp((0 - 1im)*t*ω_d - 0.5(τ^2)*(ω_d^2)) + (0.0 + 2560.0im)*g*ϵ_d*(ω_d^6)*⟨σ_{1}12⟩*exp((0 + 1im)*t*ω_d - 0.5(τ^2)*(ω_d^2)) + (0.0 - 105984.0im)*g*ϵ_d*(ω_d^8)*⟨σ_{1}21⟩*exp((0 - 1im)*t*ω_d - 0.5(τ^2)*(ω_d^2)) + (0.0 - 105984.0im)*g*ϵ_d*(ω_d^8)*⟨σ_{1}21⟩*exp((0 + 1im)*t*ω_d - 0.5(τ^2)*(ω_d^2)) + (0.0 + 105984.0im)*g*ϵ_d*(ω_d^8)*⟨σ_{1}12⟩*exp((0 - 1im)*t*ω_d - 0.5(τ^2)*(ω_d^2)) + (0.0 + 105984.0im)*g*ϵ_d*(ω_d^8)*⟨σ_{1}12⟩*exp((0 + 1im)*t*ω_d - 0.5(τ^2)*(ω_d^2)) + (0.0 - 3072.0im)*g*ϵ_d*(τ^2)*(ω_d^8)*⟨σ_{1}21⟩*exp((0 + 1im)*t*ω_d - 0.5(τ^2)*(ω_d^2)) + (0.0 - 3072.0im)*g*ϵ_d*(τ^2)*

In [28]:
eqs = complete(eqs)

MethodError: MethodError: Cannot `convert` an object of type SymbolicUtils.Term{QuantumCumulants.AvgSym, Nothing} to an object of type SymbolicUtils.Add{CNumber, Int64, Dict{Any, Number}, Nothing}
Closest candidates are:
  convert(::Type{T}, !Matched::T) where T at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/essentials.jl:218
  SymbolicUtils.Add{X, T, D, M}(::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any) where {X<:Number, T<:Number, D, M} at ~/.julia/packages/SymbolicUtils/qulQp/src/types.jl:617

In [40]:
H_test = ((9.0)ω_1*(σ_{1}22)+(-4.5 + 0.0im)ω_1 + (-4.5 + 0.0im)ω_2+(9.0 + 0.0im)ω_2*(σ_{2}22)+(-9.0 - 0.0im)g*(σ_{1}21*σ_{2}12)+(-9.0 - 0.0im)g*(σ_{1}21*σ_{2}21)+(-9.0 - 0.0im)g*(σ_{1}12*σ_{2}12)+(-9.0 - 0.0im)g*(σ_{1}12*σ_{2}21))

ErrorException: syntax: missing comma or ) in argument list